---
# MLlib Spark

## Aprendizado de Máquina no Apache Spark

Alunos:


*   **Alisson Nunes** (725862)
*   **Lucas Mathaeus** (726561)
*   **William Eugênio** (726601)

Bacharelado em Ciência da Computação (BCC)<br>
Departamento de Computação (DC)<br>
Universidade Federal de São Carlos (UFSCar)


#  (1) Obtenção dos Dados

# (2) Instalação e Configuração

In [1]:
! pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.3 MB 38 kB/s 
     |████████████████████████████████| 199 kB 48.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.0-py2.py3-none-any.whl size=281764026 sha256=07afa690d67059bc330fd825ddb97a54ea4d273ccdf0c6d299599f83b1770ec1
  Stored in directory: /root/.cache/pip/wheels/7a/8e/1b/f73a52650d2e5f337708d9f6a1750d451a7349a867f928b885
Successfully built pyspark
